### Setup libraries

In [9]:
# A dependency of the preprocessing for BERT inputs
# !pip install tf-nightly
# !pip install -q -U tensorflow-text-nightly

In [2]:
# !pip install -q tf-models-official
# !pip install tf-models-official

In [3]:
# !git clone https://github.com/tensorflow/models.git

### Import libraries

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:72.5% !important; }</style>"))

In [3]:
import os
import shutil
import re

import numpy as np
import pandas as pd

# import tensorflow as tf
# import tensorflow_hub as hub
# import tensorflow_text as text

import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, FastText, KeyedVectors
from tqdm.notebook import tqdm
# from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

In [4]:
plt.rcParams.update({'font.size': 14})
pd.set_option('precision', 3)
pd.set_option('max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('max_colwidth', 300)

### Paths to directories and files

In [5]:
DRAFT_DATASET_PATH = 'E:/kaggle/mailru/Otvety.txt'
TRAIN_DATASET_PATH = 'E:/kaggle/mailru/answers.txt'

### Loading data

In [6]:
# # Small preprocess of the answers
# question = None
# written = False

# with open(TRAIN_DATASET_PATH, 'w', encoding='utf-8') as fout:
#     with open(DRAFT_DATASET_PATH, 'r', encoding='utf-8') as fin:
#         for line in tqdm(fin):
#             if line.startswith('---'):
#                 written = False
#                 continue
#             if not written and question is not None:
#                 fout.write(question.replace('\t', ' ').strip() + '\t' + line.replace('\t', ' '))
#                 written = True
#                 question = None
#                 continue
#             if not written:
#                 question = line.strip()
#                 continue

In [7]:
# text = []

# with open(DRAFT_DATASET_PATH, 'r', encoding='utf-8') as fin:
#     for line in tqdm(fin):
#         text.append(line)        

### Preprocessing data

In [8]:
# TAG_RE = re.compile(r'<[^>]+>')
# # TAG_RE = re.compile(r'<.*?>')

# def remove_tags(text):
#     return TAG_RE.sub('', text)

In [9]:
# def preprocess_text(sen):
#     # Removing html tags
#     sentence = remove_tags(sen)

#     # Remove punctuations and numbers
#     sentence = re.sub('[^А-Яа-я0-9-ё,.!?]', ' ', sentence)

#     # Single character removal
# #     sentence = re.sub(r'\s+[А-Яа-я0-9]\s+', ' ', sentence)

#     # Removing multiple spaces
# #     sentence = re.sub(r'\s+', ' ', sentence)

#     sentence = re.sub(':\)', '', sentence)
#     sentence = re.sub('[)"]', '', sentence)  
#     sentence = re.sub('\s*\?\s*\.', '?', sentence)
#     sentence = re.sub('\s*\!\s*\.', '!', sentence)
#     sentence = re.sub('\s*\.', '.', sentence)
#     sentence = re.sub('\.+', '.', sentence)
#     sentence = re.sub('\s*\.\s*\.', ' ', sentence)
#     sentence = re.sub('\  +', ' ', sentence)
   
#     return sentence

In [10]:
# # Preprocess for models fitting
# sentences = []

# c = 0

# with open(DRAFT_DATASET_PATH, 'r', encoding='utf-8') as file:
#     for line in tqdm(file):
#         spls = preprocess_text(line)
#         sentences.append(spls)
#         c += 1
#         if c > 500:
#             break

In [17]:
morpher = MorphAnalyzer()
sw = set(get_stop_words('ru'))
exclude = set(string.punctuation)

In [22]:
def preprocess_txt(line):
    spls = ''.join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != '']
    return spls

In [23]:
core_numbers = 4
w2v_width = 300
ft_width = 300

In [24]:
def get_w2vembed_and_ftembed(file_name):
    questions = []
    with open(file_name, 'r', encoding='utf-8') as f:
        for line in tqdm(f):
            spls = line.split('\t')
            questions.append(preprocess_txt(spls[0]))
            
    modelW2V = Word2Vec(sentences=questions, size=w2v_width, window=5, min_count=1, workers=core_numbers)

    modelW2V.wv.save_word2vec_format('word2vec_wv.bin', binary=True)
    model_w2v_wv = modelW2V.wv
    del modelW2V

    return model_w2v_wv

In [25]:
flag_w2v = False
flag_ft = False

if not flag_w2v:
    model_w2v_wv = get_w2vembed_and_ftembed(DRAFT_DATASET_PATH)

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
w2v_index = annoy.AnnoyIndex(w2v_width ,'angular')
ft_index = annoy.AnnoyIndex(ft_width ,'angular')

In [ ]:
flag_w2v_index = False
flag_ft_index = False

In [ ]:
w2v_index_file = Path('w2v_index.ann')
if w2v_index_file.is_file():
    w2v_index.load('w2v_index.ann')
    flag_w2v_index = True

ft_index_file = Path('ft_index.ann')
if ft_index_file.is_file():
    ft_index.load('ft_index.ann')
    flag_ft_index = True

https://www.analyticsvidhya.com/blog/2020/07/transfer-learning-for-nlp-fine-tuning-bert-for-text-classification/

https://stackabuse.com/text-classification-with-bert-tokenizer-and-tf-2-0-in-python/

https://www.kaggle.com/nielspace/text-classification-using-bert